In [ ]:
from time import sleep, time
import os
import json
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_transpiler_service.transpiler_service import TranspilerService

In [ ]:
load_dotenv()

QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=os.getenv("IBM_TOKEN"),
    set_as_default=True,
    overwrite=True,
)

service = QiskitRuntimeService()

In [ ]:
class Colors:
    GREEN = '\033[92m'
    BLUE = '\033[94m'
    ENDC = '\033[0m'

In [ ]:
def test(num_qubits:int, local, with_ai, without_ai, results_callback:dict):
    qc = EfficientSU2(num_qubits, entanglement="circular", reps=1).decompose()

    local_result = local.run(qc)
    local_cnots = local_result.num_nonlocal_gates()
    print(f"{Colors.BLUE}Local total CNOTs: {local_cnots}{Colors.ENDC}")

    with_ai_result = with_ai.run(qc)
    with_ai_cnots = with_ai_result.num_nonlocal_gates()
    print(f"{Colors.BLUE}With AI total CNOTs: {with_ai_cnots}{Colors.ENDC}")

    without_ai_result = without_ai.run(qc)
    without_ai_cnots = without_ai_result.num_nonlocal_gates()
    print(f"{Colors.BLUE}Without AI total CNOTs: {without_ai_cnots}{Colors.ENDC}")

    results_callback['cnots_local'].append(local_cnots)
    results_callback['cnots_ai'].append(with_ai_cnots)
    results_callback['cnots_no_ai'].append(without_ai_cnots)

In [ ]:
def test_batch(num_qubits:list[int], backend_name:str, local, with_ai, without_ai, results_callback:dict):
    print(f'{Colors.GREEN}Test using: {backend_name} backend{Colors.ENDC}')
    print(f'{Colors.GREEN}qubits: {num_qubits}{Colors.ENDC}')
    
    for nq in num_qubits:
        print(f'{Colors.GREEN}--------------------{Colors.ENDC}')
        print(f'{Colors.GREEN}testing {nq} qubits{Colors.ENDC}')
        test(nq, local, with_ai, without_ai, results_callback)

        print(f'{Colors.GREEN}sleeping for 1s{Colors.ENDC}')
        sleep(1)

In [ ]:
def get_transpilers(backend):
    backend_name = backend.name
    
    local = generate_preset_pass_manager(backend=backend, optimization_level=3)
    transpiler_ai = TranspilerService(optimization_level=3, ai="true", backend_name=backend_name)
    transpiler_no_ai = TranspilerService(optimization_level=3, ai="false", backend_name=backend_name)

    return local, transpiler_ai, transpiler_no_ai

In [ ]:
def save_results(backend_name, results, num_qubits):
    filename = f"{int(time())} - {backend_name} - {num_qubits[0]}.{num_qubits[-1]}q.json"
    print(f'{Colors.GREEN}saving data at: {filename}{Colors.ENDC}')
    
    results_copy = results.copy()
    results_copy['backend'] = backend_name
    results_copy['num_qubits'] = num_qubits
    
    
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(results_copy, file, ensure_ascii=False, indent=4)

In [ ]:
def plot_results(results, num_qubits, backend_name):
    plt.title(f'{num_qubits[0]} to {num_qubits[-1]} qubits - {backend_name} backend')

    plt.plot(num_qubits, results['cnots_ai'])
    plt.plot(num_qubits, results['cnots_no_ai'])
    plt.plot(num_qubits, results['cnots_local'])
    
    plt.xlabel("Number of qubits")
    plt.ylabel("CNOT count")
    plt.legend(["AI", "without AI", "Local"])
    
    plt.savefig(f'{backend_name}_{num_qubits[0]}_to_{num_qubits[-1]}_qubits_plot.png')
    
    plt.show()

In [ ]:
backend_name = "ibm_sherbrooke"
backend = service.backend(backend_name)
local, transpiler_ai, transpiler_no_ai = get_transpilers(backend)
num_qubits = list(range(2, 81))
results = {
    'cnots_local': [],
    'cnots_ai': [],
    'cnots_no_ai': []
}

In [ ]:
test_batch(num_qubits, backend_name, local, transpiler_ai, transpiler_no_ai, results)

In [ ]:
save_results(backend_name, results, num_qubits)

In [ ]:
plot_results(results, num_qubits, backend_name)